In [ ]:
!mkdir IBT
%cd IBT
!git clone https://github.com/ManishRoy9/MT-Preperation.git

In [ ]:
!pip3 install -r MT-Preparation/requirements.txt

In [ ]:
from google.colab import files

# Upload files
uploaded = files.upload()

In [ ]:
!ls

In [ ]:
# !python3 MT-Preparation/filter.py GNOME.as-en.en GNOME.as-en.as en as

In [ ]:
!python3 MT-Preparation/filter.py english_final.txt assamese_final.txt en as

Dataframe shape (rows, columns): (47177, 2)
--- Rows with Empty Cells Deleted	--> Rows: 47174
--- Duplicates Deleted			--> Rows: 47173
--- Source-Copied Rows Deleted		--> Rows: 47152
--- Too Long Source/Target Deleted	--> Rows: 45517
--- HTML Removed			--> Rows: 45517
--- Rows will remain in true-cased	--> Rows: 45517
--- Rows with Empty Cells Deleted	--> Rows: 45517
--- Rows Shuffled			--> Rows: 45517
--- Source Saved: Anuvaad.as-en.as-filtered.as
--- Target Saved: Anuvaad.as-en.en-filtered.en


In [ ]:
!python MT-Preparation/1-train_bpe.py  english_final.txt-filtered.en assamese_final.txt-filtered.as

In [ ]:
!ls

In [ ]:
!python3 MT-Preparation/2-subword.py source.model target.model english_final.txt-filtered.en assamese_final.txt-filtered.as

In [ ]:
# !python3 MT-Preparation/2-subword.py source.model target.model  GNOME.as-en.en-filtered.en GNOME.as-en.as-filtered.as

In [ ]:
!python MT-Preparation/train_dev_test_split.py 5500 5500 english_final.txt-filtered.en.subword assamese_final.txt-filtered.as.subword

In [ ]:
!wc -l *.subword.*

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -R /content/en/ /content/drive/MyDrive/

cp: cannot stat '/content/en/': No such file or directory


In [ ]:
%cd /content/drive/MyDrive/en/
!ls

In [ ]:
!pip3 install OpenNMT-py

In [ ]:
# Create the YAML configuration file
# On a regular machine, you can create it manually or with nano
# Note here we are using some smaller values because the dataset is small
# For larger datasets, consider increasing: train_steps, valid_steps, warmup_steps, save_checkpoint_steps, keep_checkpoint

config = '''# config.yaml


## Where the samples will be written
save_data: run

# Training files
data:
    corpus_1:
        path_src:   english_final.txt-filtered.en.subword.train
        path_tgt:   assamese_final.txt-filtered.en.subword.train
        transforms: [filtertoolong]
    valid:
        path_src: english_final.txt-filtered.en.subword.dev
        path_tgt: assamese_final.txt-filtered.en.subword.dev
        transforms: [filtertoolong]

# Vocabulary files, generated by onmt_build_vocab
src_vocab: run/source.vocab
tgt_vocab: run/target.vocab

# Vocabulary size - should be the same as in sentence piece
src_vocab_size: 50000
tgt_vocab_size: 50000

# Filter out source/target longer than n if [filtertoolong] enabled
src_seq_length: 150
src_seq_length: 150

# Tokenization options
src_subword_model: source.model
tgt_subword_model: target.model

# Where to save the log file and the output models/checkpoints
log_file: train.log
save_model: models/model.as_en

# Stop training if it does not imporve after n validations
early_stopping: 4

# Default: 1000 - Save a model checkpoint for each n
save_checkpoint_steps: 1000

# To save space, limit checkpoints to last n
# keep_checkpoint: 3

seed: 3435

# Default: 100000 - Train the model to max n steps
# Increase to 200000 or more for large datasets
# For fine-tuning, add up the required steps to the original steps
train_steps: 10000

# Default: 15000 - Run validation after n steps
valid_steps: 500

# Default: 4000 - for large datasets, try up to 8000
warmup_steps: 1000
report_every: 200

# Number of GPUs, and IDs of GPUs
world_size: 1
gpu_ranks: [0]

# Batching
bucket_size: 262144
num_workers: 0  # Default: 2, set to 0 when RAM out of memory
batch_type: "tokens"
batch_size: 4096   # Tokens per batch, change when CUDA out of memory
valid_batch_size: 2048
max_generator_batches: 2
accum_count: [4]
accum_steps: [0]

# Optimization
model_dtype: "fp16"
optim: "adam"
learning_rate: 1
# warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
label_smoothing: 0.1
param_init: 0
param_init_glorot: true
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
enc_layers: 6
dec_layers: 6
heads: 8
hidden_size: 512
word_vec_size: 512
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
attention_dropout: [0.1]
'''

with open("config.yaml", "w+") as config_yaml:
  config_yaml.write(config)

In [ ]:
!cat config.yaml

In [ ]:
!nproc --all

2


In [ ]:
!onmt_build_vocab -config config.yaml -n_sample -1 -num_threads 2

In [ ]:
!nvidia-smi -L

In [ ]:
!ls

In [ ]:
!rm -rf drive/MyDrive/en/models/

In [ ]:
!onmt_train -config config.yaml

In [ ]:
!onmt_translate -model models/model.as_en_step_5000.pt -src  english_final.txt-filtered.en.subword.test -output  assamese.as.translated -gpu 0 -min_length 1

In [ ]:
!pip3 install --upgrade -q sentencepiece

In [ ]:
!python3 MT-Preparation/3-desubword.py target.model  assamese_final.txt-filtered.en.subword.test

In [ ]:
!python3 MT-Preparation/3-desubword.py target.model assamese.as.translated

In [ ]:
!wget https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py

--2023-07-10 05:52:57--  https://raw.githubusercontent.com/ymoslem/MT-Evaluation/main/BLEU/compute-bleu.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 957 [text/plain]
Saving to: ‘compute-bleu.py’

compute-bleu.py     100%[===================>]     957  --.-KB/s    in 0s      

2023-07-10 05:52:57 (27.3 MB/s) - ‘compute-bleu.py’ saved [957/957]



In [ ]:
!pip3 install sacrebleu

In [ ]:
!python3 compute-bleu.py assamese_final.txt-filtered.en.subword.test.desubword assamese.as.translated.desubword

OPUS

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!python3 MT-Preparation/filter1.py pmindia.as-en.en en

In [ ]:
!python MT-Preparation/1-train_bpe1.py pmindia.as-en.en-filtered.en en

In [ ]:
!python3 MT-Preparation/2-subword1.py en.model pmindia.as-en.en-filtered.en

In [ ]:
!onmt_translate -model models/model.as_en_step_5000.pt -src  pmindia.as-en.en-filtered.en.subword -output  pmindia.as.translated -gpu 0 -min_length 1

In [ ]:
!python3 MT-Preparation/3-desubword.py target.model pmindia.as.translated

In [ ]:
!python3 MT-Preparation/3-desubword.py as.model pmindia.as-en.en-filtered.en.subword

In [ ]:
!head -n 10 pmindia.as.translated.desubword

In [ ]:
!head -n 10 pmindia.as-en.as-filtered.as.subword.desubword

In [ ]:
pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def filter_low_quality_backtranslations(source_texts, back_translations, threshold):
    model = SentenceTransformer('xlm-roberta-base')

    embeddings_source = model.encode(source_texts)
    embeddings_back = model.encode(back_translations)

    filtered_translations = [
        back_translations[i]
        for i in range(len(embeddings_source))
        if cosine_similarity([embeddings_source[i]], [embeddings_back[i]]) >= threshold
    ]

    return filtered_translations

with open('pmindia.as-en.en-filtered.en.subword.desubword', 'r', encoding='utf-8') as file:
    source_texts = file.read().splitlines()

with open('pmindia.as.translated.desubword', 'r', encoding='utf-8') as file:
    back_translations = file.read().splitlines()

threshold = 0.9 # Set your desired threshold

filtered_translations = filter_low_quality_backtranslations(source_texts, back_translations, threshold)
print(filtered_translations)


In [ ]:
!python3 MT-Preparation/add_tag.py pmindia.as.translated.desubword bt

In [ ]:
!python3 MT-Preparation/add_tag.py pmindia.as-en.en-filtered.en.subword.desubword bt

In [ ]:
!head -n 10 pmindia.en.translated.desubword.en

In [ ]:
!head -n 10 pmindia.as-en.as-filtered.as.subword.desubword.en

In [ ]:
# Specify the file paths
file1_path = 'Anuvaad.as-en.as-filtered.as'
file2_path = 'pmindia.as.translated.desubword.en'
merged_file_path = 'filtered_source_texts.txt'

# Read the content of file1
with open(file1_path, 'r') as file1:
    file1_content = file1.read()

# Read the content of file2
with open(file2_path, 'r') as file2:
    file2_content = file2.read()

# Concatenate the contents of file1 and file2
merged_content = file1_content + file2_content

# Write the merged content to the merged file
with open(merged_file_path, 'w') as merged_file:
    merged_file.write(merged_content)


In [ ]:
# Specify the file paths
file1_path = 'Anuvaad.as-en.en-filtered.en'
file2_path = 'pmindia.as-en.en-filtered.en.subword.desubword.en'
merged_file_path = 'filtered_translations.txt'

# Read the content of file1
with open(file1_path, 'r') as file1:
    file1_content = file1.read()

# Read the content of file2
with open(file2_path, 'r') as file2:
    file2_content = file2.read()

# Concatenate the contents of file1 and file2
merged_content = file1_content + file2_content

# Write the merged content to the merged file
with open(merged_file_path, 'w') as merged_file:
    merged_file.write(merged_content)

In [ ]:
from google.colab import files
import zipfile

# List of file paths to be included in the dataset
file_paths = ['filtered_source_texts.txt', 'filtered_translations.txt']

# Create a ZIP archive
with zipfile.ZipFile('IBT.zip', 'w') as zipf:
    for file_path in file_paths:
        zipf.write(file_path)

# Download the ZIP archive
files.download('IBT.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>